# Activation function 비교 실험

hidden layer의 activation function이 hypothesis의 형태를 어떻게 변화시키는지 확인하기 위한 예제.
어떤 과목 수강생의 학습 시간과 pass/fail여부 데이터를 학습하여 학습시간에 따른 pass/fail 예측 모형을 만든다.

activation function로 ReLU, sigmoid, tanh를 사용하여 그 예측 성능을 비교하고
만들어진 모델의 hypothesis를 시각화 하여 activation fuction에 따라 decision surface가 어떻게 형성되는지를 살펴보고자 한다.

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
# test for XOR
data = np.array([[0., 0., 0.], [1., 0., 1.], [0., 1., 1.], [1., 1., 0.]], dtype=np.float32)
x = data[:,0:2]
y = data[:,-1]

In [ ]:
# test for pass/fail
data = np.array([[2., 0.], [3., 0.], [4., 0.], [5., 1.], [7., 0.], [8., 1.], [9., 1.]], dtype=np.float32)
x = data[:,0:1]
y = data[:,-1]

In [ ]:
def xavier_init(n_inputs, n_outputs, uniform=True):
    if uniform:
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev = stddev)

In [ ]:
l1_in = 1
l1_out = 4
l2_out = 1
l3_out = 1
#l1_out = 4
#l2_out = 4
#l3_out = 1

init_min = -1.0
init_max = 1.0
#activation = tf.nn.softplus
#activation = tf.nn.elu
activation = tf.nn.relu
#activation = tf.nn.sigmoid
#activation = tf.nn.tanh

with tf.name_scope('model'):
    X = tf.placeholder(dtype=tf.float32, shape=(l1_in, None), name = 'input')
    Y = tf.placeholder(dtype=tf.float32, name = 'target')
    with tf.name_scope('hidden1'):
        i1 = tf.reshape(X, [l1_in,-1])
        W1 = tf.get_variable("w_l1", shape=[l1_out, l1_in], initializer = xavier_init(l1_in, l1_out))
        b1 = tf.get_variable("b_l1", shape=[l1_out, ], initializer = xavier_init(l1_out, l1_out))
        b1 = tf.reshape(b1, [l1_out, -1])
        l1 = activation(tf.matmul(W1, i1) + b1, name = 'o_l1')
    with tf.name_scope('hidden2'):
        W2 = tf.get_variable("w_l2", shape=[l2_out, l1_out], initializer = xavier_init(l1_out, l2_out))
        b2 = tf.get_variable("b_l2", shape=[l2_out, ], initializer = xavier_init(l2_out, l2_out))
        b2 = tf.reshape(b2, [l2_out, -1])
        #l2 = activation(tf.matmul(W2, l1) + b2, name = 'o_l2') # 2-hidden layer
        l2 = tf.matmul(W2, l1) + b2
    ''' # 2-hidden layer
    with tf.name_scope('hidden3'):
        W3 = tf.get_variable("w_l3", shape=[l3_out, l2_out], initializer = xavier_init(l2_out, l3_out))
        b3 = tf.get_variable("b_l3", shape=[l3_out, ], initializer = xavier_init(l3_out, l3_out))
        l3 = tf.matmul(W3, l2) + b3
    '''
    with tf.name_scope('output'):
        #output = tf.nn.sigmoid(l3) # 2-hidden layer
        output = tf.nn.sigmoid(l2)
        predict = tf.floor(output + 0.5)

In [ ]:
with tf.name_scope('eval'):
    #loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = l3, labels = Y, name = 'loss')) # 2-hidden layer
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = l2, labels = Y, name = 'loss'))
    opt = tf.train.GradientDescentOptimizer(learning_rate=0.1)
    train = opt.minimize(loss)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(Y, predict), 'float'))
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('loss', loss)

In [ ]:
%rm -rf log_dir

In [ ]:
writer = tf.summary.FileWriter("log_dir")
summary_op = tf.summary.merge_all()

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
writer.add_graph(sess.graph)

In [ ]:
for step in range(50000):
    summary_val, loss_val, _ = sess.run([summary_op, loss, train], feed_dict={X:x.transpose(), Y:y})
    
    writer.add_summary(summary_val, step)
    if (step % 1000 == 0):
        print('step:', step, 'loss:', loss_val)
w1_val, b1_val, w2_val, b2_val = sess.run([W1, b1, W2, b2])
print('w1', w1_val, 'b1', b1_val, 'w2', w2_val, 'b2', b2_val)

In [ ]:
print(sess.run(predict, feed_dict={X:x.transpose()}))
print(sess.run(accuracy, feed_dict={X:x.transpose(), Y:y}))

In [ ]:
sess.close()

In [ ]:
#!tensorboard --log=log_dir # tensor board

In [ ]:
tf.reset_default_graph()

# 결론

neuron 4개인 hidden layer 1개 모델로하고 sigmoid, relu 두가지 activation function을 사용해서 데이터를 학습했다.

prediction결과 바로 전의 weight sum값인 network output으로 그래프를 그려보면 다음을 알 수 있다.

* activation function을 사용하지 않으면 선형 그래프가 그려진다. 따라서 5.0, 7.0, 8.0 지점을 제대로 학습할 방법은 없다.
* relu를 사용했을때는 학습 결과에 따라서 그래프에 꺾이는 지점이 생기면서 비선형 데이터를 학습할 수 있게 된다. 예제의 accuracy는 0.857143이다.
* sigmoid를 사용하면 5.0과 8.0에서 양수, 7.0에서 음수인 그래프가 만들어진다. 트레이닝 데이터에 잘 피팅했는데, 만약 테스트 데이터가 있었다면 오버피팅 한 것인지도 알 수 있었을 것이다.

no activation으로는 선형으로 회귀되지 않는 복잡한 데이터를 학습할 수 없다.


## relu 실행 결과(sample)

```
w1 [[-1.16754508], [-0.99653947], [-0.53801292], [-0.13635476]]
b1 [[ 5.52169132], [ 4.71158028], [-0.47386345], [ 0.65750885]]
w2 [[-5.15378523, -4.41914129, -0.66349363, -0.62637734]]
b2 [[ 1.0973109]]

accuracy: 0.857143

net = max(-1.16754508x + 5.52169132, 0) * -5.15378523 + max(-0.99653947x + 4.71158028, 0) * -4.41914129 + max(-0.53801292x + -0.47386345, 0) * -0.66349363 + max(-0.13635476x + 0.65750885, 0) * -0.62637734 + 1.0973109

output = 1/(1+e^(-(max(-1.16754508x + 5.52169132, 0) * -5.15378523 + max(-0.99653947x + 4.71158028, 0) * -4.41914129 + max(-0.53801292x + -0.47386345, 0) * -0.66349363 + max(-0.13635476x + 0.65750885, 0) * -0.62637734 + 1.0973109)))
```

## sigmoid 실행 결과(sample)

```
w1 [[-0.54390049], [-4.1883111 ], [-2.6911881 ], [-1.91450322]]
b1 [[  2.90393567], [ 17.9513855 ], [ 20.4703083 ], [ 11.04335976]]
w2 [[  3.61458278 -13.50610161 -12.49948788  10.69765663]]
b2 [[ 5.68671894]]

accuracy: 1.0

net = 1/(1+e^(-(-0.54390049x+ 2.90393567))) * 3.61458278 + 1/(1+e^(-(-4.1883111x+ 17.9513855))) * -13.50610161 + 1/(1+e^(-(-2.6911881x+ 20.4703083))) * -12.49948788 + 1/(1+e^(-(-1.91450322x+ 11.04335976))) * 10.69765663 + 5.68671894

output = 1/(1+e^(-(1/(1+e^(-(-0.54390049x+ 2.90393567))) * 3.61458278 + 1/(1+e^(-(-4.1883111x+ 17.9513855))) * -13.50610161 + 1/(1+e^(-(-2.6911881x+ 20.4703083))) * -12.49948788 + 1/(1+e^(-(-1.91450322x+ 11.04335976))) * 10.69765663 + 5.68671894)))
```
